In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

## First Look

In [2]:
df0 = pd.read_csv("scout_car_clean.csv")
df = df0.copy()
df.sample(3).T

,142,4753,6003
make_model,Audi A1,Audi A3,Opel Astra
body_type,Sedans,Compact,Station wagon
price,18200,21500,11850
vat,Discount,Discount,Discount
km,16500,4103,97000
prev_owner,1.0,NaN,NaN
hp,113.986878,113.986878,134.102209
Type,Used,Used,Used
Warranty_months,12.0,24.0,12.0
Body Color,Colored,Colored,Colored


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15918 entries, 0 to 15917
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   make_model             15918 non-null  object 
 1   body_type              15858 non-null  object 
 2   price                  15918 non-null  int64  
 3   vat                    11405 non-null  object 
 4   km                     15918 non-null  int64  
 5   prev_owner             9090 non-null   float64
 6   hp                     15830 non-null  float64
 7   Type                   15916 non-null  object 
 8   Warranty_months        4853 non-null   float64
 9   Body Color             15321 non-null  object 
 10  Upholstery             12198 non-null  object 
 11  Nr. of Doors           15706 non-null  float64
 12  Nr. of Seats           14941 non-null  float64
 13  Gearing Type           15918 non-null  object 
 14  Displacement_cc        15422 non-null  float64
 15  We

In [4]:
df.isna().sum() / df.shape[0]*100

make_model                0.000000
body_type                 0.376932
price                     0.000000
vat                      28.351552
km                        0.000000
prev_owner               42.894836
hp                        0.552833
Type                      0.012564
Warranty_months          69.512502
Body Color                3.750471
Upholstery               23.369770
Nr. of Doors              1.331826
Nr. of Seats              6.137706
Gearing Type              0.000000
Displacement_cc           3.115969
Weight_kg                43.805754
Fuel                      0.000000
Consumption_comb         17.433095
Gears                    29.595427
Age                       0.000000
Comfort & Convenience     5.779621
Entertainment & Media     8.631738
Extras                   18.607865
Safety & Security         6.169117
dtype: float64

## Functions

In [5]:
def infos(df, x):
    print("Column name  :  "+x)
    print("-----------------------------")
    print("Number of nulls  :  "+str(df[x].isna().sum()))
    print("% of nulls       :  "+str(round(df[x].isna().sum()/df.shape[0]*100,2))+"%")
    print("Number of uniques:  "+str(df[x].nunique()))
    print("-----------------------------\nValue Counts\n")
    print(df[x].value_counts(dropna=False))

In [6]:
def fill_mode(df, g, c):
    df[c] = df[c].fillna(df.groupby(g)[c].transform(lambda x: x.mode()[0]))
    print("Number of NaN :",df[c].isna().sum())
    print("---------------------")
    print(df[c].value_counts(dropna=False))


In [7]:
def fill_mode2(df, col1, col2, x):
    for g1 in list(df[col1].unique()):
        for g2 in list(df[col2].unique()):
            cond1 = df[col1] == g1
            cond2 = df[col2] == g2
            try:
                df.loc[cond1&cond2, x] = df.loc[cond1&cond2, x].fillna(df[cond1&cond2][x].mode()[0])
            except:
                df.loc[cond1, x] = df.loc[cond1, x].fillna(df[cond1][x].mode()[0])
    
    print("Number of NaN : ",df[x].isna().sum())
    print("------------------")
    print(df[x].value_counts(dropna=False))

In [8]:
df["Weight_kg"].isna().sum() == 6973

True

In [9]:
def fill_mean(df, col1, col2, x):
    df[x].fillna(df.groupby([col1,col2])[x].transform("mean"), inplace=True)
    if df[x].isna().sum() != 0:
        df[x].fillna(df.groupby(col2)[x].transform("mean"), inplace=True)
    print("Number of NaN : ",df[x].isna().sum())
    print("------------------")
    print(df[x].value_counts(dropna=False))    

## Handling with Categorical Features

In [10]:
categoric = [i for i in df.columns if df[i].dtype == "object"]
dfc = df[categoric]

In [11]:
dfc.isna().sum()

make_model                  0
body_type                  60
vat                      4513
Type                        2
Body Color                597
Upholstery               3720
Gearing Type                0
Fuel                        0
Comfort & Convenience     920
Entertainment & Media    1374
Extras                   2962
Safety & Security         982
dtype: int64

### body_type

In [12]:
infos(dfc,"body_type")

Column name  :  body_type
-----------------------------
Number of nulls  :  60
% of nulls       :  0.38%
Number of uniques:  9
-----------------------------
Value Counts

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           55
Coupe              25
Convertible         8
Name: body_type, dtype: int64


In [13]:
dfc.body_type.replace("Other", np.nan,inplace=True)

In [14]:
fill_mode(dfc, "make_model", "body_type")

Number of NaN : 0
---------------------
Sedans           8005
Station wagon    3678
Compact          3242
Van               817
Transporter        88
Off-Road           55
Coupe              25
Convertible         8
Name: body_type, dtype: int64


### Type

In [15]:
infos(dfc,"Type")

Column name  :  Type
-----------------------------
Number of nulls  :  2
% of nulls       :  0.01%
Number of uniques:  2
-----------------------------
Value Counts

Used    12902
New      3014
NaN         2
Name: Type, dtype: int64


In [16]:
df[df["Type"].isna()].T

,2764,5236
make_model,Audi A3,Audi A3
body_type,Sedans,Sedans
price,17900,25400
vat,NaN,NaN
km,115137,0
prev_owner,NaN,NaN
hp,147.51243,113.986878
Type,NaN,NaN
Warranty_months,NaN,NaN
Body Color,White,Colored


In [17]:
dfc.loc[(dfc["Type"].isna() == True)&(df.km>200), "Type"] = "Used"
dfc.loc[(dfc["Type"].isna() == True)&(df.km<=200), "Type"] = "New"

In [18]:
infos(dfc,"Type")

Column name  :  Type
-----------------------------
Number of nulls  :  0
% of nulls       :  0.0%
Number of uniques:  2
-----------------------------
Value Counts

Used    12903
New      3015
Name: Type, dtype: int64


### Body Color

In [19]:
infos(dfc, "Body Color")

Column name  :  Body Color
-----------------------------
Number of nulls  :  597
% of nulls       :  3.75%
Number of uniques:  2
-----------------------------
Value Counts

Colored    11915
White       3406
NaN          597
Name: Body Color, dtype: int64


In [20]:
fill_mode2(dfc, "make_model", "body_type", "Body Color")

Number of NaN :  0
------------------
Colored    12512
White       3406
Name: Body Color, dtype: int64


### vat

In [21]:
dfc.vat = df.vat

In [22]:
infos(dfc, "vat")

Column name  :  vat
-----------------------------
Number of nulls  :  4513
% of nulls       :  28.35%
Number of uniques:  1
-----------------------------
Value Counts

Discount    11405
NaN          4513
Name: vat, dtype: int64


In [23]:
dfc.vat.fillna("Non-Discount", inplace=True)
infos(dfc, "vat")

Column name  :  vat
-----------------------------
Number of nulls  :  0
% of nulls       :  0.0%
Number of uniques:  2
-----------------------------
Value Counts

Discount        11405
Non-Discount     4513
Name: vat, dtype: int64


In [24]:
dfc.isna().sum()

make_model                  0
body_type                   0
vat                         0
Type                        0
Body Color                  0
Upholstery               3720
Gearing Type                0
Fuel                        0
Comfort & Convenience     920
Entertainment & Media    1374
Extras                   2962
Safety & Security         982
dtype: int64

### Upholstery

In [25]:
infos(dfc, "Upholstery")

Column name  :  Upholstery
-----------------------------
Number of nulls  :  3720
% of nulls       :  23.37%
Number of uniques:  3
-----------------------------
Value Counts

Cloth      8422
NaN        3720
Leather    2508
Other      1268
Name: Upholstery, dtype: int64


In [26]:
fill_mode2(dfc, "make_model", "body_type", "Upholstery")

Number of NaN :  0
------------------
Cloth      11520
Leather     3083
Other       1315
Name: Upholstery, dtype: int64


### Comfort & Convenience

In [27]:
infos(dfc, "Comfort & Convenience")

Column name  :  Comfort & Convenience
-----------------------------
Number of nulls  :  920
% of nulls       :  5.78%
Number of uniques:  6198
-----------------------------
Value Counts

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  920
Air conditioning|Electrical side mirrors|Hill Holder|Power windows                                                                                                                                                                                                                                                                  

In [28]:
fill_mode2(dfc, "make_model", "body_type", "Comfort & Convenience")

Number of NaN :  0
------------------
Air conditioning|Electrical side mirrors|Hill Holder|Power windows                                                                                                                                                                                                                                                                                                                                                                                                                                   389
Air conditioning|Armrest|Automatic climate control|Cruise control|Electrical side mirrors|Leather steering wheel|Light sensor|Lumbar support|Multi-function steering wheel|Navigation system|Park Distance Control|Parking assist system sensors front|Parking assist system sensors rear|Power windows|Rain sensor|Seat heating|Start-stop system                                                                                                                                       

### Entertainment & Media

In [29]:
infos(dfc, "Entertainment & Media")

Column name  :  Entertainment & Media
-----------------------------
Number of nulls  :  1374
% of nulls       :  8.63%
Number of uniques:  346
-----------------------------
Value Counts

NaN                                                                                                 1374
Bluetooth|Hands-free equipment|On-board computer|Radio|USB                                          1282
Bluetooth|Hands-free equipment|MP3|On-board computer|Radio|USB                                       982
Bluetooth|CD player|Hands-free equipment|MP3|On-board computer|Radio|USB                             782
On-board computer|Radio                                                                              487
                                                                                                    ... 
Bluetooth|CD player|Digital radio|Hands-free equipment|Radio                                           1
CD player|Hands-free equipment|MP3|Radio|USB                                  

In [30]:
fill_mode2(dfc, "make_model", "body_type", "Entertainment & Media")

Number of NaN :  0
------------------
Bluetooth|Hands-free equipment|On-board computer|Radio|USB                      1742
Bluetooth|Hands-free equipment|MP3|On-board computer|Radio|USB                  1138
Bluetooth|CD player|Hands-free equipment|MP3|On-board computer|Radio|USB        1009
On-board computer                                                                611
Radio                                                                            558
                                                                                ... 
Bluetooth|CD player|MP3                                                            1
CD player|USB                                                                      1
Bluetooth|CD player|Digital radio|Radio|USB                                        1
Bluetooth|CD player|Digital radio|MP3|On-board computer|Radio|Television|USB       1
Hands-free equipment|On-board computer|Radio|Sound system                          1
Name: Entertainment & Media

### Safety & Security

In [31]:
infos(dfc, "Safety & Security")

Column name  :  Safety & Security
-----------------------------
Number of nulls  :  982
% of nulls       :  6.17%
Number of uniques:  4443
-----------------------------
Value Counts

NaN                                                                                                                                                                                                                                                                                                                                                               982
ABS|Central door lock|Daytime running lights|Driver-side airbag|Electronic stability control|Fog lights|Immobilizer|Isofix|Passenger-side airbag|Power steering|Side airbag|Tire pressure monitoring system|Traction control                                                                                                                                      538
ABS|Central door lock|Daytime running lights|Driver-side airbag|Electronic stability control|Immobili

In [32]:
fill_mode2(dfc, "make_model", "body_type", "Safety & Security")

Number of NaN :  0
------------------
ABS|Central door lock|Daytime running lights|Driver-side airbag|Electronic stability control|Fog lights|Immobilizer|Isofix|Passenger-side airbag|Power steering|Side airbag|Tire pressure monitoring system|Traction control                                                                                                                                      730
ABS|Central door lock|Daytime running lights|Driver-side airbag|Electronic stability control|Immobilizer|Isofix|Passenger-side airbag|Power steering|Side airbag|Tire pressure monitoring system|Traction control                                                                                                                                                 480
ABS|Central door lock|Daytime running lights|Driver-side airbag|Electronic stability control|Fog lights|Immobilizer|Isofix|LED Daytime Running Lights|Passenger-side airbag|Power steering|Side airbag|Tire pressure monitoring system|Traction contro

### Extras

In [33]:
infos(dfc, "Extras")

Column name  :  Extras
-----------------------------
Number of nulls  :  2962
% of nulls       :  18.61%
Number of uniques:  659
-----------------------------
Value Counts

Alloy wheels                                                                                               3244
NaN                                                                                                        2962
Alloy wheels|Touch screen                                                                                   697
Alloy wheels|Voice Control                                                                                  577
Alloy wheels|Touch screen|Voice Control                                                                     541
                                                                                                           ... 
Alloy wheels|Catalytic Converter|Shift paddles|Sport package|Sport seats|Sport suspension|Voice Control       1
Alloy wheels|Catalytic Converter|Roof rack|

In [34]:
fill_mode2(dfc, "make_model", "body_type", "Extras")

Number of NaN :  0
------------------
Alloy wheels                                                                                               5793
Alloy wheels|Touch screen                                                                                   697
Roof rack                                                                                                   596
Alloy wheels|Voice Control                                                                                  582
Alloy wheels|Touch screen|Voice Control                                                                     544
                                                                                                           ... 
Alloy wheels|Catalytic Converter|Shift paddles|Sport package|Sport seats|Sport suspension|Voice Control       1
Alloy wheels|Catalytic Converter|Roof rack|Sport package|Sport seats|Trailer hitch                            1
Alloy wheels|Catalytic Converter|Ski bag|Sport package|Voice Contr

In [35]:
dfc.isna().sum()

make_model               0
body_type                0
vat                      0
Type                     0
Body Color               0
Upholstery               0
Gearing Type             0
Fuel                     0
Comfort & Convenience    0
Entertainment & Media    0
Extras                   0
Safety & Security        0
dtype: int64

In [36]:
for i in dfc.columns:
    df[i] = dfc[i]

In [37]:
df.isna().sum()

make_model                   0
body_type                    0
price                        0
vat                          0
km                           0
prev_owner                6828
hp                          88
Type                         0
Warranty_months          11065
Body Color                   0
Upholstery                   0
Nr. of Doors               212
Nr. of Seats               977
Gearing Type                 0
Displacement_cc            496
Weight_kg                 6973
Fuel                         0
Consumption_comb          2775
Gears                     4711
Age                          0
Comfort & Convenience        0
Entertainment & Media        0
Extras                       0
Safety & Security            0
dtype: int64

## Handling with Numerical Features

In [38]:
df.sample(3).T

,15180,663,9985
make_model,Renault Espace,Audi A1,Opel Corsa
body_type,Van,Sedans,Sedans
price,20999,12000,16450
vat,Discount,Non-Discount,Discount
km,54000,57100,100
prev_owner,NaN,2.0,1.0
hp,158.240607,93.871546,88.507458
Type,Used,Used,New
Warranty_months,NaN,NaN,NaN
Body Color,Colored,Colored,Colored


### Nr. of Doors

In [39]:
infos(df, "Nr. of Doors")

Column name  :  Nr. of Doors
-----------------------------
Number of nulls  :  212
% of nulls       :  1.33%
Number of uniques:  6
-----------------------------
Value Counts

5.0    11574
4.0     3079
3.0      832
2.0      219
NaN      212
1.0        1
7.0        1
Name: Nr. of Doors, dtype: int64


In [40]:
df["Nr. of Doors"].replace(1,np.nan, inplace=True)
df["Nr. of Doors"].replace(7,np.nan, inplace=True)

In [41]:
infos(df, "Nr. of Doors")

Column name  :  Nr. of Doors
-----------------------------
Number of nulls  :  214
% of nulls       :  1.34%
Number of uniques:  4
-----------------------------
Value Counts

5.0    11574
4.0     3079
3.0      832
2.0      219
NaN      214
Name: Nr. of Doors, dtype: int64


In [42]:
fill_mode(df, "body_type", "Nr. of Doors")

Number of NaN : 0
---------------------
5.0    11787
4.0     3079
3.0      833
2.0      219
Name: Nr. of Doors, dtype: int64


### "Nr. of Seats"

In [43]:
infos(df, "Nr. of Seats")

Column name  :  Nr. of Seats
-----------------------------
Number of nulls  :  977
% of nulls       :  6.14%
Number of uniques:  6
-----------------------------
Value Counts

5.0    13335
4.0     1125
NaN      977
7.0      362
2.0      116
6.0        2
3.0        1
Name: Nr. of Seats, dtype: int64


In [44]:
df["Nr. of Seats"].replace(6,np.nan, inplace=True)
df["Nr. of Seats"].replace(3,np.nan, inplace=True)
infos(df, "Nr. of Seats")

Column name  :  Nr. of Seats
-----------------------------
Number of nulls  :  980
% of nulls       :  6.16%
Number of uniques:  4
-----------------------------
Value Counts

5.0    13335
4.0     1125
NaN      980
7.0      362
2.0      116
Name: Nr. of Seats, dtype: int64


In [45]:
fill_mode(df, "body_type", "Nr. of Seats")

Number of NaN : 0
---------------------
5.0    14314
4.0     1126
7.0      362
2.0      116
Name: Nr. of Seats, dtype: int64


In [46]:
df.isna().sum()

make_model                   0
body_type                    0
price                        0
vat                          0
km                           0
prev_owner                6828
hp                          88
Type                         0
Warranty_months          11065
Body Color                   0
Upholstery                   0
Nr. of Doors                 0
Nr. of Seats                 0
Gearing Type                 0
Displacement_cc            496
Weight_kg                 6973
Fuel                         0
Consumption_comb          2775
Gears                     4711
Age                          0
Comfort & Convenience        0
Entertainment & Media        0
Extras                       0
Safety & Security            0
dtype: int64

### Gears 

In [47]:
infos(df, "Gears")

Column name  :  Gears
-----------------------------
Number of nulls  :  4711
% of nulls       :  29.6%
Number of uniques:  10
-----------------------------
Value Counts

6.0     5822
NaN     4711
5.0     3239
7.0     1908
8.0      224
9.0        6
1.0        2
3.0        2
4.0        2
2.0        1
50.0       1
Name: Gears, dtype: int64


In [48]:
df.loc[(df["Gears"]<5)|(df["Gears"]>8), "Gears"] = np.nan
infos(df, "Gears")

Column name  :  Gears
-----------------------------
Number of nulls  :  4725
% of nulls       :  29.68%
Number of uniques:  4
-----------------------------
Value Counts

6.0    5822
NaN    4725
5.0    3239
7.0    1908
8.0     224
Name: Gears, dtype: int64


In [49]:
fill_mode2(df, "make_model", "body_type", "Gears")

Number of NaN :  0
------------------
6.0    8419
5.0    5367
7.0    1908
8.0     224
Name: Gears, dtype: int64


In [50]:
df.isna().sum()

make_model                   0
body_type                    0
price                        0
vat                          0
km                           0
prev_owner                6828
hp                          88
Type                         0
Warranty_months          11065
Body Color                   0
Upholstery                   0
Nr. of Doors                 0
Nr. of Seats                 0
Gearing Type                 0
Displacement_cc            496
Weight_kg                 6973
Fuel                         0
Consumption_comb          2775
Gears                        0
Age                          0
Comfort & Convenience        0
Entertainment & Media        0
Extras                       0
Safety & Security            0
dtype: int64

### Weight_kg 

In [51]:
infos(df, "Weight_kg")

Column name  :  Weight_kg
-----------------------------
Number of nulls  :  6973
% of nulls       :  43.81%
Number of uniques:  434
-----------------------------
Value Counts

NaN       6973
1163.0     574
1360.0     356
1165.0     301
1335.0     242
          ... 
1960.0       1
1258.0       1
1167.0       1
1331.0       1
2037.0       1
Name: Weight_kg, Length: 435, dtype: int64


In [52]:
df["Weight_kg"].describe()

count    8945.000000
mean     1351.106540
std       220.661532
min         1.000000
25%      1165.000000
50%      1288.000000
75%      1487.000000
max      2471.000000
Name: Weight_kg, dtype: float64

In [53]:
df["Weight_kg"].sort_values().head(10)

8354       1.0
12778      1.0
1143     102.0
8856     840.0
14312    900.0
13512    977.0
13554    977.0
14478    980.0
14653    980.0
13608    980.0
Name: Weight_kg, dtype: float64

In [54]:
df.loc[df["Weight_kg"]<800, "Weight_kg"] = np.nan
df["Weight_kg"].describe()

count    8942.000000
mean     1351.548200
std       219.375045
min       840.000000
25%      1166.000000
50%      1288.000000
75%      1487.000000
max      2471.000000
Name: Weight_kg, dtype: float64

In [55]:
fill_mean(df, "make_model", "body_type", "Weight_kg")

Number of NaN :  0
------------------
1395.477301    991
1624.584882    855
1175.411263    733
1397.233846    635
1259.453175    622
              ... 
2295.000000      1
1668.000000      1
1428.000000      1
2110.000000      1
2037.000000      1
Name: Weight_kg, Length: 460, dtype: int64


###  Displacement_cc

In [56]:
infos(df, "Displacement_cc")

Column name  :  Displacement_cc
-----------------------------
Number of nulls  :  496
% of nulls       :  3.12%
Number of uniques:  77
-----------------------------
Value Counts

1598.0    4760
999.0     2438
1398.0    1314
1399.0     749
1229.0     677
          ... 
122.0        1
1198.0       1
1195.0       1
2967.0       1
1800.0       1
Name: Displacement_cc, Length: 78, dtype: int64


In [57]:
df["Displacement_cc"].describe()

count    15422.000000
mean      1423.529568
std        333.538513
min          1.000000
25%       1229.000000
50%       1461.000000
75%       1598.000000
max      16000.000000
Name: Displacement_cc, dtype: float64

In [58]:
df["Displacement_cc"].sort_values().head(60)

13903      1.0
12834      1.0
14313      1.0
14314      1.0
14318      1.0
14319      1.0
14320      1.0
14321      1.0
14323      1.0
14324      1.0
14325      1.0
14326      1.0
14283      1.0
14327      1.0
12822      1.0
12767      1.0
12768      1.0
12814      1.0
12780      1.0
12782      1.0
12785      1.0
12669      1.0
12789      1.0
12799      1.0
12797      1.0
12749      1.0
12847      1.0
12795      1.0
12466      1.0
12500      1.0
12603      1.0
12553      1.0
12498      1.0
12472      1.0
12483      1.0
12848      1.0
14197      2.0
10558      2.0
13911     54.0
10193    122.0
7811     139.0
7810     139.0
7809     139.0
7808     139.0
8127     140.0
15576    160.0
15578    160.0
15574    160.0
15573    160.0
15577    160.0
15575    160.0
13652    890.0
14539    898.0
14537    898.0
14535    898.0
13786    898.0
14551    898.0
14552    898.0
14534    898.0
13442    898.0
Name: Displacement_cc, dtype: float64

In [59]:
df.loc[df["Displacement_cc"]<898, "Displacement_cc"] = np.nan
df["Displacement_cc"].describe()

count    15370.000000
mean      1428.165973
std        324.328817
min        898.000000
25%       1229.000000
50%       1461.000000
75%       1598.000000
max      16000.000000
Name: Displacement_cc, dtype: float64

In [60]:
fill_mean(df, "make_model", "body_type", "Displacement_cc")

Number of NaN :  0
------------------
1598.0     4760
999.0      2438
1398.0     1314
1399.0      749
1229.0      677
           ... 
1195.0        1
2967.0        1
1856.0        1
16000.0       1
1800.0        1
Name: Displacement_cc, Length: 94, dtype: int64


In [61]:
df.isna().sum()

make_model                   0
body_type                    0
price                        0
vat                          0
km                           0
prev_owner                6828
hp                          88
Type                         0
Warranty_months          11065
Body Color                   0
Upholstery                   0
Nr. of Doors                 0
Nr. of Seats                 0
Gearing Type                 0
Displacement_cc              0
Weight_kg                    0
Fuel                         0
Consumption_comb          2775
Gears                        0
Age                          0
Comfort & Convenience        0
Entertainment & Media        0
Extras                       0
Safety & Security            0
dtype: int64

### hp 

In [62]:
infos(df, "hp")

Column name  :  hp
-----------------------------
Number of nulls  :  88
% of nulls       :  0.55%
Number of uniques:  80
-----------------------------
Value Counts

113.986878    2541
88.507458     2122
108.622789    1402
134.102209    1308
147.512430    1112
              ... 
112.645856       1
261.499308       1
59.004972        1
320.504280       1
12.069199        1
Name: hp, Length: 81, dtype: int64


In [63]:
df["hp"].describe()

count    15830.000000
mean       118.503318
std         35.987628
min          1.341022
25%         88.507458
50%        113.986878
75%        135.443231
max        394.260494
Name: hp, dtype: float64

In [64]:
df["hp"].sort_values().head(30)

9284      1.341022
10578     1.341022
10467     1.341022
9288      1.341022
9287      1.341022
7018      1.341022
7017      1.341022
7015      1.341022
7013      1.341022
6945      1.341022
9289      1.341022
6004      1.341022
9291      1.341022
6033      1.341022
9286      1.341022
9298      1.341022
15154     1.341022
15155     1.341022
10579     1.341022
10609     1.341022
13884     5.364088
3912     12.069199
13758    53.640884
8856     53.640884
8407     59.004972
9740     68.392127
8692     68.392127
8693     68.392127
8714     68.392127
8694     68.392127
Name: hp, dtype: float64

In [65]:
df.loc[df["hp"]<50, "hp"] = np.nan
df["hp"].describe()

count    15808.000000
mean       118.665440
std         35.748998
min         53.640884
25%         88.507458
50%        113.986878
75%        135.443231
max        394.260494
Name: hp, dtype: float64

In [66]:
fill_mean(df, "make_model", "body_type", "hp")

Number of NaN :  0
------------------
113.986878    2541
88.507458     2122
108.622789    1402
134.102209    1308
147.512430    1112
              ... 
100.576657       1
83.088426        1
164.945717       1
320.504280       1
59.004972        1
Name: hp, Length: 93, dtype: int64


### Consumtion_comb 

In [68]:
infos(df, "Consumption_comb")

Column name  :  Consumption_comb
-----------------------------
Number of nulls  :  2775
% of nulls       :  17.43%
Number of uniques:  76
-----------------------------
Value Counts

NaN      2775
3.90      732
4.00      712
5.40      663
5.10      630
         ... 
4.55        1
43.00       1
9.10        1
11.00       1
7.50        1
Name: Consumption_comb, Length: 77, dtype: int64


In [69]:
df["Consumption_comb"].describe()

count    13143.000000
mean         4.849330
std          1.666756
min          0.000000
25%          4.000000
50%          4.700000
75%          5.400000
max         55.000000
Name: Consumption_comb, dtype: float64

In [70]:
df["Consumption_comb"].sort_values().head(20)

8216     0.0
10565    0.0
14901    1.0
13677    1.2
2619     3.0
11003    3.0
10959    3.0
10917    3.0
8730     3.0
8761     3.0
53       3.0
6745     3.0
6439     3.0
8804     3.0
13146    3.0
8821     3.0
1025     3.0
6403     3.0
10739    3.0
11014    3.0
Name: Consumption_comb, dtype: float64

In [71]:
df["Consumption_comb"].sort_values(ascending=False).head(20)

11953    55.0
13178    54.0
2196     51.0
2237     51.0
2210     51.0
2194     51.0
14583    50.0
15066    46.0
500      43.0
4088     40.0
4101     40.0
4557     38.0
4555     38.0
14529    33.0
14130    32.0
535      13.8
10512    11.0
14897    10.0
14896    10.0
14914    10.0
Name: Consumption_comb, dtype: float64

In [72]:
df.loc[df["Consumption_comb"]<3, "Consumption_comb"] = np.nan
df.loc[df["Consumption_comb"]>14, "Consumption_comb"] = np.nan
df["Consumption_comb"].describe()

count    13124.000000
mean         4.804903
std          0.939043
min          3.000000
25%          4.000000
50%          4.700000
75%          5.400000
max         13.800000
Name: Consumption_comb, dtype: float64

In [73]:
fill_mean(df, "make_model", "body_type", "Consumption_comb")

Number of NaN :  0
------------------
3.900     732
4.000     712
5.400     663
5.100     630
4.400     595
         ... 
4.345       1
13.800      1
9.100       1
11.000      1
7.500       1
Name: Consumption_comb, Length: 88, dtype: int64


In [74]:
df.isna().sum()

make_model                   0
body_type                    0
price                        0
vat                          0
km                           0
prev_owner                6828
hp                           0
Type                         0
Warranty_months          11065
Body Color                   0
Upholstery                   0
Nr. of Doors                 0
Nr. of Seats                 0
Gearing Type                 0
Displacement_cc              0
Weight_kg                    0
Fuel                         0
Consumption_comb             0
Gears                        0
Age                          0
Comfort & Convenience        0
Entertainment & Media        0
Extras                       0
Safety & Security            0
dtype: int64

### Warranty_months 

In [75]:
infos(df, "Warranty_months")

Column name  :  Warranty_months
-----------------------------
Number of nulls  :  11065
% of nulls       :  69.51%
Number of uniques:  41
-----------------------------
Value Counts

NaN     11065
12.0     2594
24.0     1118
60.0      401
36.0      279
48.0      149
6.0       125
72.0       59
3.0        33
23.0       11
18.0       10
20.0        7
25.0        6
2.0         5
50.0        4
26.0        4
16.0        4
4.0         3
1.0         3
19.0        3
34.0        3
13.0        3
28.0        2
22.0        2
14.0        2
11.0        2
46.0        2
21.0        2
9.0         2
17.0        2
45.0        2
33.0        1
40.0        1
65.0        1
10.0        1
15.0        1
7.0         1
8.0         1
56.0        1
49.0        1
47.0        1
30.0        1
Name: Warranty_months, dtype: int64


In [76]:
df.drop("Warranty_months", axis=1, inplace=True)

In [77]:
df.sample(3).T

,1665,5379,15032
make_model,Audi A1,Audi A3,Renault Espace
body_type,Sedans,Sedans,Van
price,21900,24500,24999
vat,Discount,Discount,Discount
km,28,0,5600
prev_owner,1.0,NaN,2.0
hp,88.507458,113.986878,158.240607
Type,New,New,Used
Body Color,Colored,Colored,Colored
Upholstery,Cloth,Cloth,Cloth


### prev_owner 

In [79]:
infos(df, "prev_owner")

Column name  :  prev_owner
-----------------------------
Number of nulls  :  6828
% of nulls       :  42.89%
Number of uniques:  4
-----------------------------
Value Counts

1.0    8293
NaN    6828
2.0     778
3.0      17
4.0       2
Name: prev_owner, dtype: int64


In [86]:
df.drop("prev_owner", axis=1, inplace=True)

## Final

In [87]:
df.shape

(15918, 22)

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15918 entries, 0 to 15917
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   make_model             15918 non-null  object 
 1   body_type              15918 non-null  object 
 2   price                  15918 non-null  int64  
 3   vat                    15918 non-null  object 
 4   km                     15918 non-null  int64  
 5   hp                     15918 non-null  float64
 6   Type                   15918 non-null  object 
 7   Body Color             15918 non-null  object 
 8   Upholstery             15918 non-null  object 
 9   Nr. of Doors           15918 non-null  float64
 10  Nr. of Seats           15918 non-null  float64
 11  Gearing Type           15918 non-null  object 
 12  Displacement_cc        15918 non-null  float64
 13  Weight_kg              15918 non-null  float64
 14  Fuel                   15918 non-null  object 
 15  Co

In [89]:
df.to_csv("scout_car_filled.csv", index=False)